In [2]:
import os
import sys
import logging
from typing import List, Dict, Optional
import requests
import json
import pandas as pd
from dotenv import load_dotenv
from functools import lru_cache
from datetime import datetime, timedelta

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.FileHandler('amadeus_offers.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

class AmadeusOfferFinder:
    def __init__(self, airports_data_path: str):
        """
        Initialize the Amadeus Offer Finder with configuration and data loading
        
        :param airports_data_path: Path to the airports data file
        """
        # Load environment variables
        load_dotenv(r'C:\Users\alexa\Python Projects\Offer Website\API.env')
        
        # Validate configuration
        self._validate_config()
        
        # Load airports data
        self.airports_df = self._load_airports_data(airports_data_path)
        
        # Initialize caching for API responses
        self.offer_cache = {}
        
        # Authenticate and get access token
        self.access_token = self._get_access_token()

    def _validate_config(self):
        """
        Validate required environment variables
        """
        required_vars = ['AMADEUS_CLIENT_ID', 'AMADEUS_CLIENT_SECRET']
        for var in required_vars:
            if not os.getenv(var):
                logger.error(f"Missing required environment variable: {var}")
                raise ValueError(f"Please set the {var} environment variable")

    def _load_airports_data(self, data_path: str) -> pd.DataFrame:
        """
        Load and preprocess airports data
        
        :param data_path: Path to airports data file
        :return: Processed DataFrame with airport information
        """
        try:
            columns = ["id", "name", "city", "country", "iata_code", "icao_code", 
                       "latitude", "longitude", "altitude", "timezone_offset", 
                       "DST", "timezone_name", "type", "source"]
            df = pd.read_csv(data_path, names=columns, delimiter=',')
            
            # Validate DataFrame
            required_columns = ['city', 'iata_code', 'name']
            if not all(col in df.columns for col in required_columns):
                raise ValueError("Missing required columns in airports data")
            
            return df[required_columns].dropna()
        
        except FileNotFoundError:
            logger.error(f"Airports data file not found at {data_path}")
            raise
        except pd.errors.EmptyDataError:
            logger.error("The airports data file is empty")
            raise
        except Exception as e:
            logger.error(f"Error loading airports data: {e}")
            raise

    def _get_access_token(self) -> str:
        """
        Retrieve access token from Amadeus API
        
        :return: Access token string
        """
        url = 'https://test.api.amadeus.com/v1/security/oauth2/token'
        data = {
            'grant_type': 'client_credentials',
            'client_id': os.getenv('AMADEUS_CLIENT_ID'),
            'client_secret': os.getenv('AMADEUS_CLIENT_SECRET')
        }
        try:
            response = requests.post(url, data=data)
            response.raise_for_status()
            token = response.json().get('access_token')
            logger.info("Successfully obtained access token")
            return token
        except requests.RequestException as e:
            logger.error(f"Token retrieval failed: {e}")
            raise RuntimeError("Unable to obtain access token")

    @lru_cache(maxsize=128)
    def get_iata_codes_by_city(self, city_name: str) -> List[str]:
        """
        Find IATA codes for a given city
        
        :param city_name: Name of the city
        :return: List of IATA codes
        """
        try:
            codes = self.airports_df[
                self.airports_df['city'].str.lower() == city_name.lower()
            ]['iata_code'].dropna().unique()
            
            return list(codes) if len(codes) > 0 else []
        except Exception as e:
            logger.error(f"Error finding IATA codes for {city_name}: {e}")
            return []

    @lru_cache(maxsize=128)
    def get_iata_codes_by_name(self, airport_name: str) -> List[str]:
        """
        Find IATA codes for a given airport name
        
        :param airport_name: Name of the airport
        :return: List of IATA codes
        """
        try:
            codes = self.airports_df[
                self.airports_df['name'].str.lower() == airport_name.lower()
            ]['iata_code'].dropna().unique()
            
            return list(codes) if len(codes) > 0 else []
        except Exception as e:
            logger.error(f"Error finding IATA codes for {airport_name}: {e}")
            return []

    def get_airport_name_by_iata(self, iata_code: str) -> str:
        """
        Get airport name for a given IATA code
        
        :param iata_code: IATA airport code
        :return: Airport name
        """
        try:
            name = self.airports_df[
                self.airports_df['iata_code'].str.upper() == iata_code.upper()
            ]['name'].iloc[0] if not self.airports_df[
                self.airports_df['iata_code'].str.upper() == iata_code.upper()
            ].empty else "IATA code not found"
            return name
        except Exception as e:
            logger.error(f"Error finding airport name for {iata_code}: {e}")
            return "IATA code not found"

    def fetch_flight_offers(self, origin: str, max_price: float) -> List[Dict]:
        destination_url = 'https://test.api.amadeus.com/v1/shopping/flight-destinations'
        headers = {
            'Authorization': f'Bearer {self.access_token}'
        }
        params = {
            'origin': origin,
            'maxPrice': max_price
        }

        try:
            # Add more detailed logging
            #logger.info(f"API Request - URL: {destination_url}")
            #logger.info(f"Headers: {headers}")
            #logger.info(f"Params: {params}")

            response = requests.get(destination_url, headers=headers, params=params)
        
            # Log the full response
            #logger.info(f"Response Status Code: {response.status_code}")
            #logger.info(f"Response Content: {response.text}")

            response.raise_for_status()
        
            offers = response.json().get('data', [])
        
            return offers
    
        except requests.RequestException as e:
            logger.error(f"Error fetching flight offers: {e}")
            logger.error(f"Full Error Response: {response.text if 'response' in locals() else 'No response'}")
            return []

    def display_offers(self, offers: List[Dict]):
        """
        Display flight offers in a formatted manner
        
        :param offers: List of flight offers
        """
        if not offers:
            print("No flight offers found.")
            return

        print("\n===== Available Flight Offers =====")
        for offer in offers:
            destination = offer.get('destination', 'N/A')
            price = offer.get('price', {}).get('total', 'N/A')
            departure_date = offer.get('departureDate', 'N/A')
            return_date = offer.get('returnDate', 'N/A')
            airport_name = self.get_airport_name_by_iata(destination)

            print(f"Destination: {destination} ({airport_name})")
            print(f"Price: ${price}")
            print(f"Departure Date: {departure_date}")
            print(f"Return Date: {return_date}")
            print("-----------------------------------")

def main():
    """
    Main function to orchestrate the flight offer search process
    """
    try:
        # Paths and configurations
        data_path = "C:/Users/alexa/Python Projects/OfferFligthApp/airports.dat"
        
        # Initialize offer finder
        offer_finder = AmadeusOfferFinder(data_path)
        
        # User input with validation
        while True:
            city_or_airport = input("Select search type (city/airport): ").lower()
            
            if city_or_airport not in ['city', 'airport']:
                print("Invalid selection. Please choose 'city' or 'airport'.")
                continue
            
            try:
                max_price = int(input("Enter max price for offers ($): "))
            except ValueError:
                print("Invalid price. Please enter a numeric value.")
                continue
            
            if city_or_airport == 'city':
                city_name = input("Enter origin city name: ")
                iata_codes = offer_finder.get_iata_codes_by_city(city_name)
            else:
                airport_name = input("Enter airport name: ")
                iata_codes = offer_finder.get_iata_codes_by_name(airport_name)
            
            if not iata_codes:
                print("No IATA codes found. Please try again.")
                continue
            
            # Display available airports
            print("\nAvailable Airports:")
            for code in iata_codes:
                print(f"{code}: {offer_finder.get_airport_name_by_iata(code)}")
            
            origin = input("Enter the IATA code you want to use: ").upper()
            
            if origin not in iata_codes:
                print("Invalid IATA code. Please try again.")
                continue
            
            # Fetch and display offers
            offers = offer_finder.fetch_flight_offers(origin, max_price)
            offer_finder.display_offers(offers)
            
            # Ask to continue or exit
            continue_search = input("Do you want to search again? (yes/no): ").lower()
            if continue_search != 'yes':
                break
        
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()

2024-11-25 07:45:06,538 - INFO: Successfully obtained access token


Select search type (city/airport):  city
Enter max price for offers ($):  350
Enter origin city name:  Madrid



Available Airports:
\N: Winnipeg / St. Andrews Airport
MAD: Adolfo Suárez Madrid–Barajas Airport
TOJ: Torrejón Airport
ECV: Cuatro Vientos Airport


Enter the IATA code you want to use:  MAD



===== Available Flight Offers =====
Destination: PMI (Palma De Mallorca Airport)
Price: $56.49
Departure Date: 2024-12-06
Return Date: 2024-12-10
-----------------------------------
Destination: LIS (Humberto Delgado Airport (Lisbon Portela Airport))
Price: $58.59
Departure Date: 2025-01-17
Return Date: 2025-01-24
-----------------------------------
Destination: OPO (Francisco de Sá Carneiro Airport)
Price: $64.03
Departure Date: 2024-12-07
Return Date: 2024-12-14
-----------------------------------
Destination: RAK (Menara Airport)
Price: $65.17
Departure Date: 2025-03-30
Return Date: 2025-04-04
-----------------------------------
Destination: LPA (Gran Canaria Airport)
Price: $71.89
Departure Date: 2024-12-10
Return Date: 2024-12-24
-----------------------------------
Destination: FCO (Leonardo da Vinci–Fiumicino Airport)
Price: $75.66
Departure Date: 2024-12-09
Return Date: 2024-12-17
-----------------------------------
Destination: LGW (London Gatwick Airport)
Price: $88.15
Depart

Do you want to search again? (yes/no):  no
